# Exploring Hacker News Posts
Hacker News is a site started by the startup incubator Y Combinator, where user-submitted posts are voted and commented upon. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

We're specifically interested in posts whose titles begin with either *Ask HN* or *Show HN*. 
- Users submit Ask HN posts to ask the Hacker News community a specific question.
- Users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting.

We'll compare these two types of posts to determine the following:

- Do Ask HN or Show HN receive more comments on average?
- Do posts created at a certain time receive more comments on average?
- Determine if show or ask posts receive more points on average.
- Determine if posts created at a certain time are more likely to receive more points.

Dataset can be found [here](https://www.kaggle.com/hacker-news/hacker-news-posts)

## Exploring the Data

In [1]:
from csv import reader
import datetime as dt

hn = list(reader(open('hacker_news.csv')))
hn[0]

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']

In [2]:
hn[1:5]

[['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01']]

Notice that the first list in the inner lists contains the column headers, and the lists after contain the data for one row. In order to analyze our data, we need to first remove the row containing the column headers. Let's remove that first row next.

In [3]:
headers = hn[0]
hn = hn[1:]
print(headers)
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


Now that we've removed the headers from hn, we're ready to filter our data. Since we're only concerned with post titles beginning with Ask HN or Show HN, we'll create new lists of lists containing just the data for those titles.

In [4]:
# Separating posts beginning with Ask HN and Show HN 
# (and case variations) into two different lists next.

ask_posts, show_posts, other_posts = [], [], []

for row in hn:
    title = row[1].lower()
    if title.startswith('ask hn'):
        ask_posts.append(row)
    elif title.startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
'Ask HN: {} Show HN: {} Other: {}'.format(len(ask_posts), len(show_posts),
                                          len(other_posts))

'Ask HN: 1744 Show HN: 1162 Other: 17194'

## Do Ask HN or Show HN receive more comments on average?

Next, let's determine if ask posts or show posts receive more comments on average.

In [5]:
total_ask_comments = 0
for post in ask_posts:
    num_comments = post[4]
    total_ask_comments += int(num_comments)
avg_ask_comments = round(total_ask_comments / len(ask_posts))
print(f'Average number of comments in Ask HN posts: {avg_ask_comments}')

Average number of comments in Ask HN posts: 14


In [6]:
total_show_comments = 0
for post in show_posts:
    num_comments = post[4]
    total_show_comments += int(num_comments)
avg_show_comments = round(total_show_comments / len(show_posts))
print(f'Average number of comments in Show HN posts: {avg_show_comments}')

Average number of comments in Show HN posts: 10


Therefore, people tend to commend more on Ask HN posts. This is probably because Ask HN posts invite people to answer a question, encouraging people to comment.

Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

## Do posts created at a certain time receive more comments on average?
Next, we'll determine if ask posts created at a certain time are more likely to attract comments

In [7]:
# Calculate the amount of ask posts created in each hour of the day,
# along with the number of comments received.
results_list = []
for post in ask_posts:
    created_at = post[6]
    num_comments = int(post[4])
    results_list.append([created_at, num_comments])
    
counts_by_hour, comments_by_hour = {}, {}
for row in results_list:
    created_at = row[0]
    num_comments = row[1]
    # Extract hour from post
    dt_obj = dt.datetime.strptime(created_at, '%m/%d/%Y %H:%M')
    hour = dt_obj.strftime('%H')
    if hour in counts_by_hour:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += num_comments
    else:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = num_comments

We created two dictionaries:

- `counts_by_hour`: contains the number of ask posts created during each hour of the day.
- `comments_by_hour`: contains the corresponding number of comments ask posts created at each hour received.

Next, we'll use these two dictionaries to calculate the average number of comments for posts created during each hour of the day.

In [8]:
avg_by_hour = []

for hour in counts_by_hour:
    avg = comments_by_hour[hour] / counts_by_hour[hour]
    avg_by_hour.append([hour, avg])
avg_by_hour

[['09', 5.5777777777777775],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['14', 13.233644859813085],
 ['16', 16.796296296296298],
 ['23', 7.985294117647059],
 ['12', 9.41095890410959],
 ['17', 11.46],
 ['15', 38.5948275862069],
 ['21', 16.009174311926607],
 ['20', 21.525],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532],
 ['03', 7.796296296296297],
 ['05', 10.08695652173913],
 ['19', 10.8],
 ['01', 11.383333333333333],
 ['22', 6.746478873239437],
 ['08', 10.25],
 ['04', 7.170212765957447],
 ['00', 8.127272727272727],
 ['06', 9.022727272727273],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034]]

Although we now have the results we need, this format makes it hard to identify the hours with the highest values. Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [9]:
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
print(swap_avg_by_hour)

sorted_swap = sorted(swap_avg_by_hour, reverse = True)
print('Top 5 Hours for Ask Posts Comments')
for row in sorted_swap[:5]:
    dt_obj = dt.datetime.strptime(row[1], '%H')
    template = '{}: {:.2f} average comments per post'
    print(template.format(dt_obj.strftime('%H'), row[0]))

[[5.5777777777777775, '09'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [16.796296296296298, '16'], [7.985294117647059, '23'], [9.41095890410959, '12'], [11.46, '17'], [38.5948275862069, '15'], [16.009174311926607, '21'], [21.525, '20'], [23.810344827586206, '02'], [13.20183486238532, '18'], [7.796296296296297, '03'], [10.08695652173913, '05'], [10.8, '19'], [11.383333333333333, '01'], [6.746478873239437, '22'], [10.25, '08'], [7.170212765957447, '04'], [8.127272727272727, '00'], [9.022727272727273, '06'], [7.852941176470588, '07'], [11.051724137931034, '11']]
Top 5 Hours for Ask Posts Comments
15: 38.59 average comments per post
02: 23.81 average comments per post
20: 21.52 average comments per post
16: 16.80 average comments per post
21: 16.01 average comments per post


Thus, the best hours to create a post are 2 a.m, 3 p.m., 4 p.m., 8 p.m., and 9 p.m. (US Eastern Time)

## Determine if show or ask posts receive more points on average
Now let's see whether Ask or Show posts get more points on average.

In [10]:
def get_avg_points(posts):
    total_points = 0
    for row in posts:
        points = int(row[3])
        total_points += points
    return total_points / len(posts)

ask_points = get_avg_points(ask_posts)
show_points = get_avg_points(show_posts)

f'Avg Points: Ask ({ask_points:2f}) Show ({show_points:2f}))'

'Avg Points: Ask (15.061927) Show (27.555077))'

Thus, Show HN posts are rated higher on average. This is probably because Show HN posts display something of interest that users might like. Ask HN posts, on the other hand, ask people to answer a question, which is why they attract more comments than points.

## Determine if posts created at a certain time are more likely to receive more points
Now let's see if posts created at a certain time are more likely to receive more points.

In [11]:
def get_points_by_hour(posts):
    points_by_hour = {}
    for post in posts:
        points = int(post[3])
        created_at = post[-1]
        dt_obj = dt.datetime.strptime(created_at, '%m/%d/%Y %H:%M')
        hour = dt_obj.strftime('%H')
        points_by_hour[hour] = points_by_hour.get(hour, 0) + points
    return sorted(points_by_hour.items(), key = lambda x : x[1], reverse = True)

ask_pbh = get_points_by_hour(ask_posts)
show_pbh = get_points_by_hour(show_posts)

In [12]:
ask_pbh

[('15', 3479),
 ('16', 2522),
 ('13', 2062),
 ('17', 1941),
 ('18', 1741),
 ('21', 1721),
 ('19', 1513),
 ('14', 1282),
 ('20', 1151),
 ('10', 1102),
 ('11', 825),
 ('02', 793),
 ('12', 782),
 ('01', 700),
 ('06', 591),
 ('23', 581),
 ('05', 552),
 ('08', 515),
 ('22', 511),
 ('00', 451),
 ('04', 389),
 ('03', 374),
 ('07', 361),
 ('09', 329)]

In [13]:
show_pbh

[('16', 2634),
 ('12', 2543),
 ('17', 2521),
 ('13', 2438),
 ('15', 2228),
 ('18', 2215),
 ('14', 2187),
 ('22', 1856),
 ('20', 1819),
 ('19', 1702),
 ('23', 1526),
 ('11', 1480),
 ('00', 1173),
 ('21', 866),
 ('01', 700),
 ('10', 681),
 ('03', 679),
 ('09', 553),
 ('08', 519),
 ('07', 494),
 ('04', 386),
 ('06', 375),
 ('02', 340),
 ('05', 104)]

So, the best times to create an Ask HN post are 1 p.m. to 9 p.m. On the other hand, Show HN posts receive the most points in between 12 p.m. to 12 a.m.

# Conclusion
Do Ask HN or Show HN receive more comments on average?
- Do posts created at a certain time receive more comments on average?
- Determine if show or ask posts receive more points on average.
- Determine if posts created at a certain time are more likely to receive more points.

We started off with four questions in mind. We wanted to know which type of posts receive more comments. We found that Ask HN posts get more comments.

Next, we wanted to determine the best times to create a post to attract the most number of comments and highest number of points. We found that commenting during the day i.e. during p.m. hours is the best time to make a post.

We were also able to determine that Show HN posts get more points than Ask HN posts on average.